In [5]:
from PIL import Image
import pandas as pd
import os, cv2
import numpy as np

In [ ]:
!pip install Ninja

In [ ]:
!git clone https://github.com/PeikeLi/Self-Correction-Human-Parsing.git

In [ ]:
!git clone https://github.com/Healthcare-Robotics/bodies-at-rest.git

In [9]:
!mkdir Self-Correction-Human-Parsing-Segmented-Dataset
!mkdir Self-Correction-Human-Parsing-Dataset

In [10]:
%cd Self-Correction-Human-Parsing

/content/Self-Correction-Human-Parsing


In [11]:
!mkdir input
!mkdir output
!mkdir dataset

In [12]:
%cd /content/bodies-at-rest/PressurePose

/content/bodies-at-rest/PressurePose


In [ ]:
!chmod +x download_real.sh
!bash download_real.sh

In [ ]:
object = pd.read_pickle(r'/content/bodies-at-rest/data_BR/real/S103/prescribed.p')
print(len(object['RGB']))
print(object['RGB'][0].shape)

In [33]:
path = "/content/bodies-at-rest/data_BR/real/"
folders = os.listdir(path)
w, h = 880, 440

for i, folder in enumerate(folders):
  object = pd.read_pickle(r'/content/bodies-at-rest/data_BR/real/'+ folder +'/prescribed.p')
  for j in range(0, len(object['RGB'])):
    rgb_img = np.zeros((h, w, 3), dtype=np.uint8)
    rgb_img = object['RGB'][j]
    rgb_img = Image.fromarray(rgb_img, 'RGB')
    rgb_img.save(f'/content/Self-Correction-Human-Parsing-Dataset/{i}{j}.jpg')

In [34]:
!python '/content/Self-Correction-Human-Parsing/simple_extractor.py' --dataset 'pascal' --model-restore '/content/drive/MyDrive/models/exp-schp-201908270938-pascal-person-part.pth' --input-dir '/content/Self-Correction-Human-Parsing-Dataset/' --output-dir '/content/Self-Correction-Human-Parsing-Segmented-Dataset'

Evaluating total class number 7 with ['Background', 'Head', 'Torso', 'Upper Arms', 'Lower Arms', 'Upper Legs', 'Lower Legs']
100% 914/914 [03:53<00:00,  3.91it/s]


In [35]:
path = "/content/Self-Correction-Human-Parsing-Segmented-Dataset/"
segmented_images = os.listdir(path)

len(segmented_images)

914

In [ ]:
%cd /content/Self-Correction-Human-Parsing/dataset/
!mkdir train_images
!mkdir train_segmentation
!mkdir val_images
!mkdir val_segmentation

In [37]:
train_id = open('train_id.txt', 'w')
val_id = open('val_id.txt', 'w')

for i ,image in enumerate(segmented_images):
  img = cv2.imread(os.path.join(path, image))
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  img_gray[img_gray > 0] = 255
  img_gray[img_gray == 0] = 0
  image_name = image.split('.')[0]
  if i < 800:
    cv2.imwrite(f"train_images/{image_name}.JPG", img_gray)
    train_id.write(f"{image_name}")
    train_id.write('\n')
  else:
    cv2.imwrite(f"val_images/{image_name}.JPG", img_gray)
    val_id.write(f"{image_name}")
    val_id.write('\n')

train_id.close()
val_id.close()

In [38]:
path = "/content/Self-Correction-Human-Parsing-Segmented-Dataset/"
segmented_images = os.listdir(path)

for i, image in enumerate(segmented_images):
  img_rgb = cv2.imread(os.path.join(path, image))
  img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

  img_gray[img_gray == 0] = 0
  img_gray[img_gray == 15] = 1
  img_gray[img_gray == 38] = 2
  img_gray[img_gray == 53] = 3
  img_gray[img_gray == 75] = 4
  img_gray[img_gray == 90] = 5
  img_gray[img_gray == 113] = 6

  if i < 800:
    cv2.imwrite(f"train_segmentation/{image}", img_gray)
  else:
    cv2.imwrite(f"val_segmentation/{image}", img_gray)

In [ ]:
%cd /content/Self-Correction-Human-Parsing/
!mkdir pretrain_model

In [ ]:
import gdown

url = "http://sceneparsing.csail.mit.edu/model/pretrained_resnet/resnet101-imagenet.pth"
out = "/content/Self-Correction-Human-Parsing/pretrain_model/resnet101-imagenet.pth"
gdown.download(url, out) 

In [41]:
!python 'train.py' --batch-size 2 --data-dir 'dataset' --epochs 50 --num-classes 7

Namespace(arch='resnet101', batch_size=2, cycle_epochs=10, data_dir='dataset', epochs=50, eval_epochs=10, gpu='0,1,2', ignore_label=255, imagenet_pretrain='./pretrain_model/resnet101-imagenet.pth', input_size='473,473', lambda_c=0.1, lambda_e=1, lambda_s=1, learning_rate=0.007, log_dir='./log', model_restore='./log/checkpoint.pth.tar', momentum=0.9, num_classes=7, random_mirror=False, random_scale=False, schp_restore='./log/schp_checkpoint.pth.tar', schp_start=100, start_epoch=0, weight_decay=0.0005)
image mean: [0.406, 0.456, 0.485]
image std: [0.225, 0.224, 0.229]
input space:BGR
BGR Transformation
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to 